## Project Introduction

The dataset is in regards to used car sales fro eBay Kleinanzeigen, a classifieds section of the German eBay website. The dataset is dirty since it contains scrapped data. I will attempt to clean it and peform some analysis work.

In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-cars-database-50000-data-points/autos.csv


In [121]:
autos = pd.read_csv('/kaggle/input/used-cars-database-50000-data-points/autos.csv', encoding='Latin-1')

In [122]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [123]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

### We can see from the output from the cell above that: 
* the dataset contains 5000 entries and 20 columns
* most columns are of string type except for `yearOfRegistration`, `powerPS`, `monthOfRegistration`, `nrOfPictures` and `postalCode`
* some columns contain NULL values: `vehicleType`, `gearBox`, `model`, `fuelType` and `notRepairedDamage`
* inconsistent column naming 
* Some dates are identified as string values with others as numeric values

### 1. Rename columns

change all column names to be snakecase

In [124]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [125]:
autos.rename({'yearOfRegistration': 'registration_year',
             'monthOfRegistration': 'registration_month',
             'notRepairedDamage': 'unrepaired_damage',
             'dateCreated': 'date_created',
             'dateCrawled': 'date_crawled',
             'offerType': 'offer_type',
             'vehicleType': 'vehicle_type',
             'powerPS': 'power_ps',
             'fuelType': 'fuel_type',
             'nrOfPictures': 'num_pictures',
             'postalCode': 'postal_code',
             'lastSeen': 'last_seen'}, axis=1, inplace=True)
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'date_created', 'num_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

### 2. Find potentially redundant or mistyped columns 

In [126]:
# numerical columns
autos.describe()

,registration_year,power_ps,registration_month,num_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


In [127]:
# string columns
autos.describe(include='object')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,gearbox,model,odometer,fuel_type,brand,unrepaired_damage,date_created,last_seen
count,50000,50000,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2,2,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


### Observations from the output:
* a couple of columns contain almost (49999 out of 50000) the same value for all rows: `seller` and `offer_type`
* some string column can be typed to numberic: `price` and `odometer`

### Let's change the type of `price` column to float

Taking a look at the format

In [128]:
# or .unique()
autos['price'].value_counts().sort_index()

$0          1421
$1           156
$1,000       639
$1,039         1
$1,040         1
            ... 
$996           1
$998           5
$999         434
$999,990       1
$999,999       2
Name: price, Length: 2357, dtype: int64

Remove non-numeric characters

In [129]:
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '')

Change type to float

In [130]:
autos['price'] = autos['price'].astype(float)

In [131]:
# numerical columns
autos.describe()

,price,registration_year,power_ps,registration_month,num_pictures,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,9.840044e+03,2005.073280,116.355920,5.723360,0.0,50813.627300
std,4.811044e+05,105.712813,209.216627,3.711984,0.0,25779.747957
min,0.000000e+00,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1.100000e+03,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2.950000e+03,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,7.200000e+03,2008.000000,150.000000,9.000000,0.0,71540.000000
max,1.000000e+08,9999.000000,17700.000000,12.000000,0.0,99998.000000


### Do the same for `odometer` column

In [132]:
autos['odometer'].value_counts().sort_index(key=lambda x: x.str.replace('km', '').str.replace(',', '').astype(int))

5,000km        967
10,000km       264
20,000km       784
30,000km       789
40,000km       819
50,000km      1027
60,000km      1164
70,000km      1230
80,000km      1436
90,000km      1757
100,000km     2169
125,000km     5170
150,000km    32424
Name: odometer, dtype: int64

In [133]:
autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',', '')
autos['odometer'] = autos['odometer'].astype(int)

In [134]:
# add unit to the name of odometer column
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)
autos.describe()

,price,registration_year,power_ps,odometer_km,registration_month,num_pictures,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,0.0,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,0.0,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,0.0,99998.000000


### 3. Look for suspicious data 

Do this for `odometer_km` and `price` columns

### Start with `odometer_km`

In [135]:
autos['odometer_km'].unique().shape

(13,)

In [136]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [146]:
autos['odometer_km'].value_counts().sort_index()

5000        863
10000       137
20000       459
30000       496
40000       528
50000       716
60000       882
70000       966
80000      1199
90000      1526
100000     1916
125000     4834
150000    31685
Name: odometer_km, dtype: int64

The values look okay.

### Now for `price` column

In [138]:
autos['price'].unique().shape

(2357,)

In [139]:
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [140]:
autos['price'].value_counts()

0.0        1421
500.0       781
1500.0      734
2500.0      643
1200.0      639
           ... 
6202.0        1
18310.0       1
898.0         1
11240.0       1
789.0         1
Name: price, Length: 2357, dtype: int64

We'll remove some outliers

In [141]:
mid_50 = autos['price'].quantile([.25, .75])
price_25, price_75 = mid_50.iloc[0], mid_50.iloc[1]

iqr = price_75 - price_25
low = price_25 - 1.5 * iqr
high = price_75 + 1.5 * iqr

low, iqr, high

(-8050.0, 6100.0, 16350.0)

In [142]:
autos = autos.loc[autos['price'] < high]
print('Count: ', autos.shape[0], '\nMin price: ', autos['price'].min(), '\nMax price:', autos['price'].max())

Count:  46207 
Min price:  0.0 
Max price: 16333.0


We have removed entries with prices higher than $16,350.

### 4. Dealing with columns with dates
Here's some information about date columns in the dataset we know:

- `date_crawled`: added by the crawler, identified as string
- `last_seen`: added by the crawler, identified as string
- `date_created`: from the website, identified as string
- `registration_month`: from the website, identified as numeric
- `registration_year`: from the website, identified as numeric

### First take a look at the 'string dates'

In [143]:
string_dates = autos[['date_crawled', 'last_seen', 'date_created']]
string_dates.head()

,date_crawled,last_seen,date_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


### Snoop around the dates for a bit, starting with `date_created`

First thing we notice is that only the date is specified (not the time)

In [149]:
date_created = string_dates['date_created'].str[:10]
date_created.value_counts(normalize=True, dropna=False).sort_index()

2015-08-10    0.000022
2015-09-09    0.000022
2015-11-10    0.000022
2015-12-05    0.000022
2015-12-30    0.000022
                ...   
2016-04-03    0.038890
2016-04-04    0.036683
2016-04-05    0.011687
2016-04-06    0.003246
2016-04-07    0.001212
Name: date_created, Length: 73, dtype: float64

In [148]:
date_created.describe()

count          46207
unique            73
top       2016-04-03
freq            1797
Name: date_created, dtype: object

In [160]:
date_created.str[:4].astype(int).describe()

count    46207.000000
mean      2015.999892
std          0.010402
min       2015.000000
25%       2016.000000
50%       2016.000000
75%       2016.000000
max       2016.000000
Name: date_created, dtype: float64

Some observations for `date_created`:

- The ads were created from August 10th, 2015 to April 7th, 2016
- They were created on 73 different days
- The highest number of ads posted in a day during the period is 1797, on April 3rd, 2016

### Then move on to `last_seen`

In [152]:
last_seen = string_dates['last_seen'].str[:10]
last_seen.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001169
2016-03-06    0.004675
2016-03-07    0.005648
2016-03-08    0.008072
2016-03-09    0.010323
2016-03-10    0.011254
2016-03-11    0.013180
2016-03-12    0.024931
2016-03-13    0.009457
2016-03-14    0.013072
2016-03-15    0.016275
2016-03-16    0.016945
2016-03-17    0.028848
2016-03-18    0.007640
2016-03-19    0.016361
2016-03-20    0.021274
2016-03-21    0.021274
2016-03-22    0.022118
2016-03-23    0.019153
2016-03-24    0.020213
2016-03-25    0.019975
2016-03-26    0.017400
2016-03-27    0.016599
2016-03-28    0.021663
2016-03-29    0.022984
2016-03-30    0.025537
2016-03-31    0.024347
2016-04-01    0.023633
2016-04-02    0.025191
2016-04-03    0.025819
2016-04-04    0.025191
2016-04-05    0.120847
2016-04-06    0.213669
2016-04-07    0.125262
Name: last_seen, dtype: float64

In [153]:
last_seen.describe()

count          46207
unique            34
top       2016-04-06
freq            9873
Name: last_seen, dtype: object

In [162]:
last_seen.str[:4].astype(int).describe()

count    46207.0
mean      2016.0
std          0.0
min       2016.0
25%       2016.0
50%       2016.0
75%       2016.0
max       2016.0
Name: last_seen, dtype: float64

Some observations for `last_seen`:

- The ads were last viewed from March 5th, 2016 to April 7th, 2016
- They were last viewed on 34 different days, roughly within a month before April 7th, 2016 
- Most ads (around 10,000 of them) were last viewed on April 6th, 2016

### Finally we do the same with `date_crawled`

In [182]:
date_crawled = string_dates['date_crawled'].str[:10]
date_crawled.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025581
2016-03-06    0.013894
2016-03-07    0.036055
2016-03-08    0.033610
2016-03-09    0.033263
2016-03-10    0.032636
2016-03-11    0.032376
2016-03-12    0.037224
2016-03-13    0.015495
2016-03-14    0.037051
2016-03-15    0.033869
2016-03-16    0.029844
2016-03-17    0.031813
2016-03-18    0.012942
2016-03-19    0.034367
2016-03-20    0.037613
2016-03-21    0.037332
2016-03-22    0.032657
2016-03-23    0.032636
2016-03-24    0.029130
2016-03-25    0.032160
2016-03-26    0.032874
2016-03-27    0.030688
2016-03-28    0.034778
2016-03-29    0.034259
2016-03-30    0.033804
2016-03-31    0.031684
2016-04-01    0.032960
2016-04-02    0.034930
2016-04-03    0.038631
2016-04-04    0.036358
2016-04-05    0.012963
2016-04-06    0.003181
2016-04-07    0.001342
Name: date_crawled, dtype: float64

In [157]:
date_crawled.describe()

count          46207
unique            34
top       2016-04-03
freq            1785
Name: date_crawled, dtype: object

In [163]:
date_crawled.str[:4].astype(int).describe()

count    46207.0
mean      2016.0
std          0.0
min       2016.0
25%       2016.0
50%       2016.0
75%       2016.0
max       2016.0
Name: date_crawled, dtype: float64

Some observations for `date_crawled`:

- The ads were scraped from March 5th, 2016 to April 7th, 2016
- They were scraped on 34 different days, roughly within a month before April 7th, 2016 
- The highest number of ads scraped in a day during the period is 1785, on April 3rd, 2016

### Let's also take a look at one of the columns with numeric dates - `registration_year`

In [158]:
autos['registration_year'].describe()

count    46207.000000
mean      2004.342416
std         96.234804
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2007.000000
max       9999.000000
Name: registration_year, dtype: float64

In [164]:
autos['registration_year'].astype(str).describe()

count     46207
unique       88
top        2000
freq       3340
Name: registration_year, dtype: object

The first thing we notice is that there seems to be some erroneous data. The registration year can't be `1000` or `9999`. Let's do some cleaning.

In [165]:
print("Number of entries before: ", autos.shape[0])

Number of entries before:  46207


In [175]:
# Firstly, the registration year can be later than 2016
year_high = 2016

# Secondly, let's assume the earliest year in which a car can be registered was the year 1885 (when the first car was invented)
year_low = 1885

autos = autos[autos['registration_year'].between(1885, 2016)]

# Lastly, let's say an ad can't be posted before the car was registered
autos = autos[autos['registration_year'] <= autos['date_created'].str[:4].astype(int)]

print("Number of entries after: ", autos.shape[0])

Number of entries after:  44285


We've removed around 2000 entries with invalid `registration_year`

In [184]:
autos['registration_year'].describe()

count    44285.000000
mean      2002.295247
std          6.894476
min       1910.000000
25%       1999.000000
50%       2002.000000
75%       2007.000000
max       2016.000000
Name: registration_year, dtype: float64

In [186]:
autos['registration_year'].value_counts(normalize=True, dropna=False).sort_index()

1910    0.000203
1929    0.000023
1934    0.000045
1937    0.000068
1938    0.000023
          ...   
2012    0.018336
2013    0.008965
2014    0.005419
2015    0.002642
2016    0.028768
Name: registration_year, Length: 70, dtype: float64

All cars were registered from the 1910 to 2016

### 4. Next, lets find out what kinds of cars were recorded in the dataset

Specifically let's take a look at the top 6 most `brand`s in the dataset

In [215]:
top_5_brands_list = list(autos['brand'].value_counts(normalize=True).head(6).index)
top_5_brands_list

['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford']

Get the price and mileage information for each of the 5 brands

In [224]:
top_prices_mileage_info = {}


for brand in top_5_brands_list:
    mean_price = autos.loc[autos['brand'] == brand, 'price'].mean()
    min_price = autos.loc[autos['brand'] == brand, 'price'].min()
    max_price = autos.loc[autos['brand'] == brand, 'price'].max()
    mileage = autos.loc[autos['brand'] == brand, 'odometer_km'].mean()
    
    top_prices_mileage_info[brand] = ["$" + str(round(mean_price, 2)),
                                      "$" + str(round(min_price, 2)),
                                      "$" + str(round(max_price, 2)),
                                      round(mileage, 2)]
    

for key, val in top_prices_mileage_info.items():
    print(key, ":")
    print('Average price: ', val[0])
    print('Min price: ', val[1])
    print('Max price: ', val[2])
    print('Average mileage', val[3])
    print('\n')

volkswagen :
Average price:  $4038.95
Min price:  $0.0
Max price:  $16300.0
Average mileage 132694.04


opel :
Average price:  $2619.19
Min price:  $0.0
Max price:  $16300.0
Average mileage 130387.7


bmw :
Average price:  $5467.3
Min price:  $0.0
Max price:  $16333.0
Average mileage 138469.41


mercedes_benz :
Average price:  $5156.37
Min price:  $0.0
Max price:  $16250.0
Average mileage 138341.53


audi :
Average price:  $5524.82
Min price:  $0.0
Max price:  $16290.0
Average mileage 139527.62


ford :
Average price:  $2857.05
Min price:  $0.0
Max price:  $16250.0
Average mileage 126764.34




In [232]:
price_mileage = pd.DataFrame(data=top_prices_mileage_info, index=['mean_price', 'min_price', 'max_price', 'mean_mileage'])
price_mileage = price_mileage.swapaxes('index', 'columns')
price_mileage

,mean_price,min_price,max_price,mean_mileage
volkswagen,$4038.95,$0.0,$16300.0,132694
opel,$2619.19,$0.0,$16300.0,130388
bmw,$5467.3,$0.0,$16333.0,138469
mercedes_benz,$5156.37,$0.0,$16250.0,138342
audi,$5524.82,$0.0,$16290.0,139528
ford,$2857.05,$0.0,$16250.0,126764


Some observations:
- for each brand, the prices of hign end cars are roughly the same, around $16,000
- `ford` and `opel` have cheaper middle end cars and have relatively less mileage